In [1]:
# Downloading required libraries
!pip install -q transformers datasets sentencepiece

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 39.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 468.7/468.7 kB 12.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 32.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 200.1/200.1 kB 6.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 34.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 17.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 4.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.9/132.9 kB 6.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.2/212.2 kB 5.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 264.6/264.6 kB 6.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.2/114.2 kB 4.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 158.8/158.8 kB 1.6 MB/s eta 0:00:00


In [2]:
# Deleting the cache content of the transformer namespace
!rm -rf $HOME/.cache/huggingface/
!mkdir $HOME/.cache/huggingface/

In [3]:
# Downloading the pre-trained t5 model
!transformers-cli download t5-small

2023-04-16 19:36:33.651974: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
/usr/local/lib/python3.9/dist-packages/transformers/models/t5/tokenization_t5_fast.py:155: FutureWarning: This tokenizer was incorrectly instantiated with a model max length of 512 which will be corrected in Transformers v5.
For now, this behavior is kept to avoid breaking backwards compatibility when padding/encoding with `truncation is True`.
- Be aware that you SHOULD NOT rely on t5-small automatically truncating your input to 512 when padding/encoding.
- If you want to encode/pad to sequences longer than 512 you can either instantiate this tokenizer with `model_max_length` or pass `max_length` when encoding/padding.
- To avoid this warning, please instantiate this tokenizer with `model_max_length` set to your preferred value.
  warnings.warn(


In [4]:
# Load the required libraries
# from transformers import T5ForConditionalGeneration, T5Tokenizer
from transformers import T5Tokenizer, T5ForConditionalGeneration
from datasets import load_dataset
import torch
import os

In [5]:
# Load the pre-trained T5 model and tokenizer
model = T5ForConditionalGeneration.from_pretrained('t5-small')
tokenizer = T5Tokenizer.from_pretrained('t5-small')

/usr/local/lib/python3.9/dist-packages/transformers/models/t5/tokenization_t5.py:163: FutureWarning: This tokenizer was incorrectly instantiated with a model max length of 512 which will be corrected in Transformers v5.
For now, this behavior is kept to avoid breaking backwards compatibility when padding/encoding with `truncation is True`.
- Be aware that you SHOULD NOT rely on t5-small automatically truncating your input to 512 when padding/encoding.
- If you want to encode/pad to sequences longer than 512 you can either instantiate this tokenizer with `model_max_length` or pass `max_length` when encoding/padding.
- To avoid this warning, please instantiate this tokenizer with `model_max_length` set to your preferred value.
  warnings.warn(


In [6]:
# Load the SQuAD v2 dataset
dataset = load_dataset('squad_v2')

Extracting data files:   0%|          | 0/2 [00:00<?, ?it/s]

Generating train split:   0%|          | 0/130319 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/11873 [00:00<?, ? examples/s]

Dataset squad_v2 downloaded and prepared to /root/.cache/huggingface/datasets/squad_v2/squad_v2/2.0.0/09187c73c1b837c95d9a249cd97c2c3f1cebada06efe667b4427714b27639b1d. Subsequent calls will reuse this data.


  0%|          | 0/2 [00:00<?, ?it/s]

In [7]:
# Extract the passages 
passages = dataset['train']['context']

In [8]:
# Extract the answers
answers=[]
answer_array = dataset['train']['answers']
for i in answer_array:
  if(len(i['text']) != 0):
    answers.append(i['text'][0])
  else:
    answers.append("")

In [9]:
# Prepare the input and output for the T5 model
inputs = []
outputs = []
for i in range(len(passages)):
    input_text = f'generate question: "{passages[i]}"'
    output_text = f'{dataset["train"][i]["question"]}'
    inputs.append(input_text)
    outputs.append(output_text)

In [10]:
# Tokenize the input and output using the T5 tokenizer
input_ids = tokenizer(inputs, padding=True, truncation=True, return_tensors='pt')['input_ids']
output_ids = tokenizer(outputs, padding=True, truncation=True, return_tensors='pt')['input_ids']

In [11]:
# Fine-tune the T5 model on the SQuAD v2 dataset
# Checking if GPU is available and switching model to GPU Runtime
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
optimizer = torch.optim.AdamW(model.parameters(), lr=1e-4)

In [ ]:
model.train()

In [13]:
# Actual Training
# Took 3h 15m on Colab GPU
total_length_of_input = len(input_ids)
for i in range(len(input_ids)):
    input_ids_batch = input_ids[i].unsqueeze(0).to(device)
    output_ids_batch = output_ids[i].unsqueeze(0).to(device)
    loss = model(input_ids_batch, labels=output_ids_batch).loss
    loss.backward()
    optimizer.step()
    optimizer.zero_grad()
    print("Running " + str(i) + " of " + str(total_length_of_input))

Running 130300 of 130319
Running 130301 of 130319
Running 130302 of 130319
Running 130303 of 130319
Running 130304 of 130319
Running 130305 of 130319
Running 130306 of 130319
Running 130307 of 130319
Running 130308 of 130319
Running 130309 of 130319
Running 130310 of 130319
Running 130311 of 130319
Running 130312 of 130319
Running 130313 of 130319
Running 130314 of 130319
Running 130315 of 130319
Running 130316 of 130319
Running 130317 of 130319
Running 130318 of 130319


In [14]:
# Generate questions based on a given passage
sample_passage = '''Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ bee-YON-say) (born September 4, 1981) is an American singer, songwriter, record producer and actress. Born and raised in Houston, Texas, she performed in various singing and dancing competitions as a child, and rose to fame in the late 1990s as lead singer of R&B girl-group Destiny's Child. Managed by her father, Mathew Knowles, the group became one of the world's best-selling girl groups of all time. Their hiatus saw the release of Beyoncé's debut album, Dangerously in Love (2003), which established her as a solo artist worldwide, earned five Grammy Awards and featured the Billboard Hot 100 number-one singles "Crazy in Love" and "Baby Boy".'''
input_text = f'generate question: "{sample_passage}"'
input_ids = tokenizer.encode(input_text, return_tensors='pt')
output = model.generate(input_ids=input_ids, max_length=64, num_beams=4, no_repeat_ngram_size=2, early_stopping=True)
output_text = tokenizer.decode(output[0], skip_special_tokens=True)

In [16]:
# Print the generated questions
print(output_text)

Who is Beyonce?


In [17]:
# Save the fine-tuned model and tokenizer
model.save_pretrained('fine_tuned_model')
tokenizer.save_pretrained('fine_tuned_model')

('fine_tuned_model/tokenizer_config.json',
 'fine_tuned_model/special_tokens_map.json',
 'fine_tuned_model/spiece.model',
 'fine_tuned_model/added_tokens.json')